## 인스타그램 크롤링

- 입력키워드(#맛집 #성수(지역명컬럼)), 크롤링값(사진, 해시태그, 날짜, 위치, 본문, 댓글, 좋아요수)
- 해시태그 나중에 처리
- 주은 / 경아

In [68]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import pandas as pd

In [69]:
class page:
    def __init__(self, driverDir):

        options = Options()
        options.add_argument("disable-infobars")
        options.add_argument("enable-automation")
        options.add_argument('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome')
        self.driver = webdriver.Chrome(driverDir, options = options)
        
    def open(self, url):
        
        self.driver.get(url)
        time.sleep(5)
    
    def login(self, userId, userPw):
        
        idField = self.driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
        pwField = self.driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
        idField.clear()
        pwField.clear()
        idField.send_keys(userId)
        pwField.send_keys(userPw)
        time.sleep(11)
        
        loginButton = self.driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]')
        loginButton.click()
        time.sleep(11)
        
        noSaveButton = self.driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button')
        noSaveButton.click()
        time.sleep(15)
        
        noAlarmButton = self.driver.find_element_by_xpath("/html/body/div[4]/div/div/div/div[3]/button[2]")
        noAlarmButton.click()
        time.sleep(11)
        
    def search(self, tag):
        
        tagUrl = "https://www.instagram.com/explore/tags/" + tag
        self.driver.get(tagUrl)
        time.sleep(11)
        
    def openFirst(self):
        firstPost = self.driver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[1]/a')
        firstPost.click()
        time.sleep(10)
        
    def getDriver(self):
        return (self.driver)

In [74]:
class tagPage:
    def __init__(self, tagDriver):

        self.tagDriver = tagDriver
        self.df = pd.DataFrame()
        
    def getDF(self):
        return (self.df)
    
    def saveDF(fileName):
        self.df.to_excel(fileName)
        
    def moveToNext(self,):
        
        try:
            nextButton = self.tagDriver.find_element_by_xpath("/html/body/div[5]/div[1]/div/div/a[2]")
        except:
            nextButton = self.tagDriver.find_element_by_xpath("/html/body/div[5]/div[1]/div/div/a")
        nextButton.click()

    def crawl(self, number):
        
        idx = []
        imgs = []
        dates = []
        locations = []
        bodies = []
        comments = []
        likesNums = []
        
        for i in range(number):

            print(str(i) + "-th post")
            html = self.tagDriver.page_source
            soup = BeautifulSoup(html, "html.parser")
            img = soup.select_one("div.ZyFrc div.KL4Bh img")
            date = soup.select_one("div.k_Q0X time")["datetime"]
            try:
                location = soup.select_one("a.O4GlU").text
            except:
                location = ""
            body = soup.select_one("ul.XQXOT > div:nth-child(1) span:nth-child(2)").text
            comment = []
            for c in soup.select("ul.Mr508 div.C4VMK span:nth-child(2)"):
                comment.append(c.text)
            likesNum = soup.select_one("div.Nm9Fw span")

            idx.append(i)
            imgs.append(img)
            dates.append(date)
            locations.append(location)
            bodies.append(body)
            comments.append(comment)
            likesNums.append(likesNum)
            
            self.moveToNext()
            time.sleep(10)
        
        self.df["id"] = idx
        self.df["이미지"] = imgs
        self.df["날짜"] = dates
        self.df["위치"] = locations
        self.df["본문"] = bodies
        self.df["댓글"] = comments
        self.df["좋아요수"] = likesNums
        
        return (self.df)

In [75]:
userId = "아이디입력"
userPw = "비밀번호입력"

In [76]:
instaPage = page("/Users/gyeonga/Desktop/chromedriver");
instaPage.open('https://www.instagram.com/accounts/login/?next=https://www.instagram.com/&source=mobile_nav');
instaPage.login(userId, userPw)
instaPage.search("성수맛집")
instaPage.openFirst()

In [77]:
tag = tagPage(instaPage.getDriver())
tag.crawl(10)

0-th post
1-th post
2-th post
3-th post
4-th post
5-th post
6-th post
7-th post
8-th post
9-th post


,id,이미지,날짜,위치,본문,댓글,좋아요수
0,0,[],2021-03-16T08:10:45.000Z,소랑호젠_사랑하려고,🍽소랑호젠 #코밥심_성수 📍서울 성동구 와...,"[🙌, 분위기도 좋고 음식 비주얼도 너무 예쁘네요❤️, 비쥬얼이 넘 좋네요😍, 오오...","[1,611]"
1,1,[],2021-03-12T09:40:17.000Z,,🍽꾸아#코밥심_성수 📍서울시 성동구 서울수...,"[👏, 하... 가야겠어요 저곳🙌 오늘도 좋은 곳 소개 감사해용, 우와.. 😍😍😍,...","[3,101]"
2,2,[],2021-03-14T10:55:15.000Z,연무장,펜트하우스 지금 시작하면 늦었나🤦,"[😢, 늦었다고 생각할 때가 진짜 너무 늦었다, 딱 적절함 시즌2까지 달려, 행님,...",[343]
3,3,[],2021-03-10T09:29:10.000Z,서울숲,서울숲 레드브릭스모크하우스총평: 가격이 착한 바베큐집한줄평: 바베큐 어때유?.바베큐...,[오웃. 여기 앞에 지나가다 왠지모를 너낌이 있었는데 맛집너낌이었군요. 정말 옛동네...,"[1,046]"
4,4,[],2021-03-18T02:47:31.000Z,,#난포요즘 성수동에서 제일 사람 많은 식당 중 하나.무조건 오픈어택하시는 걸 추천합...,"[내 원픽은,, 묵은지말이야 여러분, @ssunmi._.v @hi_dayoung 이...","[2,767]"
5,5,[],2021-03-18T03:31:31.000Z,카츠성수,에르메스 접시가 인상적인 돈까스 집이다. 이거 깨트리면 뭔 일 생기려나 싶다. 히레...,"[근데 여섯 조각은 좀 적지 않냐.., B0, 알바 설거지 손떨려서 우에하냐ㅋㅋㅋㅋ...",[107]
6,6,[],2021-03-10T12:02:32.000Z,성수동,이집은 파스타가 존맛이다 . #여음 ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 파스타 소스가...,[여음 파스타 자극적이그 맛있도❤️ 스테이크 크림리조또도 맛있어보이네요 ⁼̴̤̆◡̶...,"[2,279]"
7,7,[],2021-03-16T11:49:15.000Z,성수동,"<버거보이, 성수>짭짤한 오리지널 치즈 버거에 매료되었다. 크리스피한 수제 패티에 ...","[🙌, 피드 구경 잘 하고갑니당먹팔 신청해요 ₍˄·͈ ‧̫ ·͈˄₎, 버거 완전 맛...",[835]
8,8,[],2021-03-18T03:25:21.000Z,성수역 어딘가,#오늘의점심 #무로정 🌺감바스에 와인한잔 했던 한식 퓨전 경한식 그집..자꼬 자꼬 ...,"[❤️, 꼭 가봐야겠네요 😍, 😍😍😍😍😍🤤🤤🤤🤤🤤, 육회요리가 완죤 고급지고 좋네여...","[1,303]"
9,9,[],2021-03-19T01:03:23.000Z,용싸롱,-성수동 용싸롱입니다.📌금일(03월 19일 ...,[❤️],[3]
